<a href="https://colab.research.google.com/github/tnusser/ImageRetrieval/blob/master/Autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports


In [40]:
import tensorflow as tf
import numpy as np
import sklearn
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization, Activation, Flatten, Conv2DTranspose, Reshape
from keras.models import Model, Sequential
import keras.backend as K 
import matplotlib.pyplot as plt

## 3.1 Dataset Preparation

In [2]:
 (raw_x_train, raw_y_train), (raw_x_test, raw_y_test) = tf.keras.datasets.cifar10.load_data()

170500096/170498071 [==============================] - 4s 0us/step


In [3]:
x_train = (raw_x_train/255).astype("float32")
x_test = (raw_x_test/255).astype("float32")

## 3.2 Building the Autoencoder

In [5]:
x_val = x_train[:3000]
x_train = x_train[3000:15000]

In [67]:
i = Input(shape=(32, 32, 3))
e1 = Conv2D(32, (3, 3), activation='relu', padding='same')(i)
e2 = MaxPooling2D(pool_size=(3, 3))(e1)
e3 = Conv2D(64, (3, 3), activation="relu")(e2)
e4 = MaxPooling2D(pool_size=(3, 3))(e3)
e5 = Flatten()(e4)
e6 = Dense(10, activation='relu')(e5)

# encoder = Model()
# encoder.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(32,32,3)))
# encoder.add(MaxPooling2D(pool_size=(3, 3)))
# encoder.add(Conv2D(64, (3, 3), activation="relu"))
# encoder.add(MaxPooling2D(pool_size=(3, 3)))
# encoder.add(Flatten())
# encoder.add(Dense(10, activation='relu'))
# encoder.summary()
# out_enc = encoder.layers[-1]

encoder = Model(i, e6, name='encoder')
encoder.summary()


# decoder = Model()
# decoder.add(Dense(conv_shape[1] * conv_shape[2] * conv_shape[3], activation='relu', input_shape=(10,)))
# decoder.add(Reshape((conv_shape[1], conv_shape[2], conv_shape[3])))
# decoder.add(Conv2DTranspose(filters=3, kernel_size=3, activation="relu", padding="same"))
# decoder.summary()

conv_shape = K.int_shape(e1)
d_i = Input(shape=(10,))
d1 = Dense(conv_shape[1] * conv_shape[2] * conv_shape[3], activation='relu',)(d_i)
d2 = Reshape((conv_shape[1], conv_shape[2], conv_shape[3]))(d1)
d3 = Conv2DTranspose(filters=3, kernel_size=3, activation="relu", padding="same")(d2)

decoder = Model(d_i, d3, name='decoder')
decoder.summary()


Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_45 (InputLayer)        (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_83 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
max_pooling2d_78 (MaxPooling (None, 10, 10, 32)        0         
_________________________________________________________________
conv2d_84 (Conv2D)           (None, 8, 8, 64)          18496     
_________________________________________________________________
max_pooling2d_79 (MaxPooling (None, 2, 2, 64)          0         
_________________________________________________________________
flatten_44 (Flatten)         (None, 256)               0         
_________________________________________________________________
dense_47 (Dense)             (None, 10)                2570

In [70]:
autoencoder_outputs = decoder(encoder(i))
autoencoder = Model(i, autoencoder, name='autoencoder')
autoencoder.summary()

Model: "autoencoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_45 (InputLayer)        (None, 32, 32, 3)         0         
_________________________________________________________________
encoder (Model)              (None, 10)                21962     
_________________________________________________________________
decoder (Model)              (None, 32, 32, 3)         361315    
Total params: 383,277
Trainable params: 383,277
Non-trainable params: 0
_________________________________________________________________


In [71]:
autoencoder.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
autoencoder.fit(x_train, x_train, validation_data=(x_test, x_test), epochs=3)

Train on 9000 samples, validate on 10000 samples
Epoch 1/3
9000/9000 [==============================] - 22s 2ms/step - loss: nan - accuracy: 0.5110 - val_loss: nan - val_accuracy: 0.5086
Epoch 2/3
9000/9000 [==============================] - 22s 2ms/step - loss: nan - accuracy: 0.5114 - val_loss: nan - val_accuracy: 0.5086
Epoch 3/3
9000/9000 [==============================] - 22s 2ms/step - loss: nan - accuracy: 0.5114 - val_loss: nan - val_accuracy: 0.5086


In [72]:
# Loss plot
history_dict = history.history
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
epochs = range(1, len(loss_values) + 1)
plt.plot(epochs, loss_values, 'bo', label='Training loss')
plt.plot(epochs, val_loss_values, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Accuracy plot
plt.clf()
acc_values = history_dict['acc']
val_acc_values = history_dict['val_acc']
plt.plot(epochs, acc_values, 'bo', label='Training acc')
plt.plot(epochs, val_acc_values, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

NameError: ignored